In [ ]:
# LTN: https://github.com/logictensornetworks/logictensornetworks/blob/master/examples/multiclass_classification/multiclass-singlelabel.ipynb
# Common.py : https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

In [ ]:
!pip install PyTDC rdkit-pypi ltn keras==2.15.0 -qq
!wget https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/

In [ ]:
!ls "/content/drive/MyDrive/Project/AI and Cardiology"

ls: cannot access '/content/drive/MyDrive/Project/AI and Cardiology': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import pandas as pd
import numpy as np
import ltn
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
tqdm.pandas()

# Utils Functions

In [ ]:
# Threshold ref: https://pubs.acs.org/doi/epdf/10.1021/acs.jcim.3c01301
def label_th(pic50):
    classes = []
    for x in pic50:
        if x>=5:
            classes.append(1)
        else:
            classes.append(0)

    return np.asarray(classes)
class_map = {
    "blocks":1,
    "non-blocks":0,
}

### Data Acquisition

In [ ]:
dataset_path = "/content/drive/MyDrive/Project/AI and Cardiology/Cardiotoxicity/Dataset"

In [ ]:
!ls "{dataset_path}/External-Data"

CDK-external_test_set_pos.csv  eval_set_herg_70.csv	       herg_mmb_emb_h70.npz
CDK-herg60.csv		       external_test_set_pos.csv       Morgan-external_test_set_pos.csv
CDK-herg70.csv		       herg_mmb_emb_external_test.npz  Morgan-herg60.csv
eval_set_herg_60.csv	       herg_mmb_emb_h60.npz	       Morgan-herg70.csv


In [ ]:
df = pd.read_csv(f"{dataset_path}/UniChemDB-Data/final-herg.csv")
df.dropna(subset = ['std_smiles'],inplace = True)
df.reset_index(drop = True,inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20409 entries, 0 to 20408
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          20388 non-null  object
 1   std_smiles  20409 non-null  object
 2   classes     20409 non-null  object
dtypes: object(3)
memory usage: 478.5+ KB


# Data



In [ ]:
#External Test-1: https://github.com/Abdulk084/CardioTox/blob/master/data/external_test_set_pos.csv
test_pos_df = pd.read_csv(f"{dataset_path}/External-Data/external_test_set_pos.csv")


In [ ]:
# External Test h70, h60 dataset: https://github.com/issararab/CToxPred/tree/main/data/raw/hERG
test_h60_df = pd.read_csv(f"{dataset_path}/External-Data/eval_set_herg_60.csv")
test_h70_df = pd.read_csv(f"{dataset_path}/External-Data/eval_set_herg_70.csv")


In [ ]:
test_h60_df.head()

,InChl Key,SMILES,Source,pIC50
0,LIHJHFVXLZSRNK-UHFFFAOYSA-N,Cn1ccc(C[N+]2=CC(c3cccc(C(F)(F)F)c3)C=N2)n1,US Patent,5.647817
1,RXGDDWPITVSKDR-UHFFFAOYSA-N,CC(C)(C)OC(=O)N1CCN(c2nc3c([N+](=O)[O-])c(Br)c...,US Patent,5.602060
2,YRSBMPKJFDYYFO-UHFFFAOYSA-N,Fc1cccc(Oc2cc(C(F)(F)F)nc(N3CCc4nc[nH]c4C3)n2)c1,US Patent,5.599980
3,OMQQLDITRYIEHZ-UHFFFAOYSA-N,Cn1nccc1Cc1cn(-c2ccc(F)c(Cl)c2)nn1,US Patent,5.364516
4,BXBUTKPGTGJGTQ-YOEHRIQHSA-N,CNC[C@@H](c1ccc(Cl)c(Cl)c1)[C@@H](OC)c1cccc(NS...,US Patent,5.327902


In [ ]:
#Threshold conversion
test_h60_df['target'] = label_th(test_h60_df.pIC50)
test_h70_df['target'] = label_th(test_h70_df.pIC50)

In [ ]:
!ls "{dataset_path}"/UniChemDB-Data

final-herg.csv	final-herg-split.csv  mmb_embeddings.npy  Morgan-unichemdb.csv


In [ ]:
X = pd.read_csv(dataset_path+f"/UniChemDB-Data/Morgan-unichemdb.csv").set_index("Name")
y = df['classes'].replace(class_map).values

In [ ]:
X.shape, y.shape

((20409, 2048), (20409,))

In [ ]:
!ls '{dataset_path}/External-Data'

CDK-external_test_set_pos.csv  eval_set_herg_70.csv	       herg_mmb_emb_h70.npz
CDK-herg60.csv		       external_test_set_pos.csv       Morgan-external_test_set_pos.csv
CDK-herg70.csv		       herg_mmb_emb_external_test.npz  Morgan-herg60.csv
eval_set_herg_60.csv	       herg_mmb_emb_h60.npz	       Morgan-herg70.csv


In [ ]:
ext_pos_df = pd.read_csv(f'{dataset_path}/External-Data/Morgan-external_test_set_pos.csv').rename(columns = {'Name':"target"})
ext_h60_df = pd.read_csv(f'{dataset_path}/External-Data/Morgan-herg60.csv').rename(columns = {'Name':"target"})
ext_h70_df = pd.read_csv(f'{dataset_path}/External-Data/Morgan-herg70.csv').rename(columns = {'Name':"target"})


In [ ]:
batch_size = 64
ds_train = tf.data.Dataset.from_tensor_slices((X,y)).batch(batch_size)
idx = X.sample(1000).index
ds_test = tf.data.Dataset.from_tensor_slices((X.iloc[idx],y[idx])).batch(batch_size)

# LTN

Predicate with softmax `P(x,class)`

In [ ]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training=False):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

logits_model = MLP(2)
p = ltn.Predicate.FromLogits(logits_model, activation_function="softmax", with_class_indexing=True)

Constants to index/iterate on the classes

In [ ]:
class_A = ltn.Constant(0, trainable=False)
class_B = ltn.Constant(1, trainable=False)
# class_C = ltn.Constant(2, trainable=False)

Operators and axioms

In [ ]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [ ]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    x_A = ltn.Variable("x_A",features[labels==0])
    x_B = ltn.Variable("x_B",features[labels==1])
    # x_C = ltn.Variable("x_C",features[labels==2])
    axioms = [
        Forall(x_A,p([x_A,class_A],training=training)),
        Forall(x_B,p([x_B,class_B],training=training)),
        # Forall(x_C,p([x_C,class_C],training=training))
    ]
    for i in range(len(axioms)):
        if tf.math.is_nan(axioms[i].tensor):
            axioms[i].tensor  =0.0
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

Initialize all layers and the static graph

In [ ]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

Initial sat level 0.50042


# Training

Define the metrics. While training, we measure:
1. The level of satisfiability of the Knowledge Base of the training data.
1. The level of satisfiability of the Knowledge Base of the test data.
3. The training accuracy.
4. The test accuracy.

In [ ]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

Define the training and test step

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['train_accuracy'](tf.one_hot(labels,2),predictions)

@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['test_accuracy'](tf.one_hot(labels,2),predictions)

Train

In [ ]:
import commons

EPOCHS = 500

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="herg_Morgan_results.csv",
    track_metrics=20
)

Epoch 0, train_sat_kb: 0.4952, test_sat_kb: 0.5789, train_accuracy: 0.6314, test_accuracy: 0.7360
Epoch 20, train_sat_kb: 0.6330, test_sat_kb: 0.6777, train_accuracy: 0.8528, test_accuracy: 0.8720
Epoch 40, train_sat_kb: 0.6893, test_sat_kb: 0.7103, train_accuracy: 0.9066, test_accuracy: 0.8980
Epoch 60, train_sat_kb: 0.7210, test_sat_kb: 0.7419, train_accuracy: 0.9270, test_accuracy: 0.9250
Epoch 80, train_sat_kb: 0.7387, test_sat_kb: 0.7563, train_accuracy: 0.9367, test_accuracy: 0.9260
Epoch 100, train_sat_kb: 0.7493, test_sat_kb: 0.7761, train_accuracy: 0.9444, test_accuracy: 0.9430
Epoch 120, train_sat_kb: 0.7575, test_sat_kb: 0.7751, train_accuracy: 0.9481, test_accuracy: 0.9370
Epoch 140, train_sat_kb: 0.7638, test_sat_kb: 0.7806, train_accuracy: 0.9506, test_accuracy: 0.9430
Epoch 160, train_sat_kb: 0.7673, test_sat_kb: 0.7929, train_accuracy: 0.9525, test_accuracy: 0.9470
Epoch 180, train_sat_kb: 0.7690, test_sat_kb: 0.8051, train_accuracy: 0.9539, test_accuracy: 0.9530
Epoch 

In [ ]:
!ls "{dataset_path}/../Model-Weights"

hERG-Karim-CDK.keras	     hERG-Karim-Morgan.keras   hERG-UniChemDB-Morgan_CDK.keras
hERG-Karim-MMB.keras	     hERG-UniChemDB-CDK.keras  hERG-UniChemDB-Morgan.keras
hERG-Karim-Morgan_CDK.keras  hERG-UniChemDB-MMB.keras


In [ ]:
tf.keras.saving.load_model(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-Morgan.keras")

TypeError: Could not locate class 'MLP'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'MLP', 'config': {'n_classes': 2}, 'registered_name': 'MLP', 'build_config': {'input_shape': [[None, 2048]]}}

In [ ]:
logits_model.save(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-Morgan.keras")
# tf.keras.backend.clear_session()
# logits_model = MLP(2)
# logits_model([ext_pos_df.drop(['target'],axis = 1).values])
# logits_model.load_weights(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-Morgan.keras")

## Model Evaluation

In [ ]:
from sklearn.metrics import (
    accuracy_score as ays,
    f1_score as fs,
    precision_score as ps,
    recall_score as rs,
    matthews_corrcoef as mcc,
    roc_auc_score as auc,
    balanced_accuracy_score,
    confusion_matrix
)

In [ ]:
def print_score(xtest,ytest,name):

    pred_test = logits_model.predict([xtest]).argmax(-1)

    auc_test = auc(ytest, pred_test)


    tn, fp, fn, tp = confusion_matrix(ytest, pred_test).ravel()

    specificity_test = tn / (tn + fp)

    sensitivity_test = tp / (tp + fn)

    NPV_test = tn / (tn + fn)

    PPV_test = tp / (tp + fp)
    Accuracy_test = ays(ytest, pred_test)
    Balanced_Accuracy_test = balanced_accuracy_score(ytest, pred_test)

    MCC_test= mcc(ytest, pred_test)

    CCR = (sensitivity_test+specificity_test)/2

    f1_score = fs(ytest, pred_test)
    print(f"MCC_test_{name}: " + str(MCC_test))
    print(f"NPV_test_{name}g: " + str(NPV_test))
    print(f"Accuracy_test_{name}: " + str(Accuracy_test))
    print(f"PPV_test_{name}: " + str(PPV_test))
    print(f"specificity_test_{name}: " + str(specificity_test))
    print(f"sensitivity_test_{name}: " + str(sensitivity_test))
    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))

    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))
    print(f"Correct Classification Rate (CCR){name}: " + str(CCR))
    print(f"F1 Score{name}: " + str(f1_score))



In [ ]:
print_score(ext_pos_df.drop("target",axis = 1),ext_pos_df['target'],'External Data Test-1 (pos)')

2/2 [==============================] - 0s 7ms/step
MCC_test_External Data Test-1 (pos): 0.8469139703212729
NPV_test_External Data Test-1 (pos)g: 0.8666666666666667
Accuracy_test_External Data Test-1 (pos): 0.9318181818181818
PPV_test_External Data Test-1 (pos): 0.9655172413793104
specificity_test_External Data Test-1 (pos): 0.9285714285714286
sensitivity_test_External Data Test-1 (pos): 0.9333333333333333
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.930952380952381
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.930952380952381
Correct Classification Rate (CCR)External Data Test-1 (pos): 0.930952380952381
F1 ScoreExternal Data Test-1 (pos): 0.9491525423728815


In [ ]:
print_score(ext_pos_df.drop("target",axis = 1),ext_pos_df['target'],'External Data Test-1 (pos)')


2/2 [==============================] - 0s 6ms/step
MCC_test_External Data Test-1 (pos): 0.8469139703212729
NPV_test_External Data Test-1 (pos)g: 0.8666666666666667
Accuracy_test_External Data Test-1 (pos): 0.9318181818181818
PPV_test_External Data Test-1 (pos): 0.9655172413793104
specificity_test_External Data Test-1 (pos): 0.9285714285714286
sensitivity_test_External Data Test-1 (pos): 0.9333333333333333
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.930952380952381
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.930952380952381
Correct Classification Rate (CCR)External Data Test-1 (pos): 0.930952380952381
F1 ScoreExternal Data Test-1 (pos): 0.9491525423728815


In [ ]:
print_score(ext_h60_df.drop("target",axis = 1),ext_h60_df['target'],'External hERG-60')


8/8 [==============================] - 0s 2ms/step
MCC_test_External hERG-60: 0.6303636116432123
NPV_test_External hERG-60g: 0.740506329113924
Accuracy_test_External hERG-60: 0.804
PPV_test_External hERG-60: 0.9130434782608695
specificity_test_External hERG-60: 0.936
sensitivity_test_External hERG-60: 0.672
Balanced_Accuracy_testExternal hERG-60: 0.804
Balanced_Accuracy_testExternal hERG-60: 0.804
Correct Classification Rate (CCR)External hERG-60: 0.804
F1 ScoreExternal hERG-60: 0.7741935483870968


In [ ]:
print_score(ext_h70_df.drop("target",axis = 1),ext_h70_df['target'],'External hERG-70')


15/15 [==============================] - 0s 4ms/step
MCC_test_External hERG-70: 0.662374324432512
NPV_test_External hERG-70g: 0.7591836734693878
Accuracy_test_External hERG-70: 0.8266384778012685
PPV_test_External hERG-70: 0.8991228070175439
specificity_test_External hERG-70: 0.8899521531100478
sensitivity_test_External hERG-70: 0.7765151515151515
Balanced_Accuracy_testExternal hERG-70: 0.8332336523125996
Balanced_Accuracy_testExternal hERG-70: 0.8332336523125996
Correct Classification Rate (CCR)External hERG-70: 0.8332336523125996
F1 ScoreExternal hERG-70: 0.8333333333333333


In [ ]:
round(0.8899,4)

0.8899